У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE, SMOTENC

In [ ]:
raw_df = pd.read_csv('/content/drive/MyDrive/Data Science/customer_segmentation_train.csv', index_col=0)

In [ ]:
raw_df.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
# Перевірка на пропущені значення
print(raw_df.isnull().sum())

# Розділення даних на вхідні та цільові змінні
X = raw_df.drop('Segmentation', axis=1)
y = raw_df['Segmentation']

# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Виявлення числових і категоріальних колонок
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(include=[object]).columns

# Обробка числових колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Обробка категоріальних колонок
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Комбінування трансформерів для числових та категоріальних колонок в один препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення пайплайну
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Тренування пайплайну на тренувальних даних
X_train_preprocessed = model_pipeline.fit_transform(X_train)
X_test_preprocessed = model_pipeline.transform(X_test)

# Перевірка оброблених даних
print("Processed training data shape:", X_train_preprocessed.shape)
print("Processed test data shape:", X_test_preprocessed.shape)

# Збереження оброблених даних у DataFrame для подальшого аналізу
X_train_processed_df = pd.DataFrame(X_train_preprocessed)
X_test_processed_df = pd.DataFrame(X_test_preprocessed)

# Перевірка оброблених даних
print(X_train_processed_df.head())
print(X_test_processed_df.head())


Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64
Processed training data shape: (6454, 28)
Processed test data shape: (1614, 28)
         0         1         2    3    4    5    6    7    8    9   ...   18  \
0 -0.331955 -0.510955  0.762766  1.0  0.0  0.0  1.0  0.0  1.0  0.0  ...  0.0   
1 -0.630627 -0.818941  1.430362  1.0  0.0  1.0  0.0  1.0  0.0  0.0  ...  0.0   
2  0.325124 -0.510955 -1.240020  1.0  0.0  0.0  1.0  1.0  0.0  0.0  ...  0.0   
3 -1.048768  1.644951  0.095171  1.0  0.0  1.0  0.0  1.0  0.0  0.0  ...  0.0   
4  0.743265  0.105018 -1.240020  1.0  0.0  1.0  0.0  0.0  1.0  0.0  ...  0.0   

    19   20   21   22   23   24   25   26   27  
0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
1  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
#Читання даних
raw_df = pd.read_csv('/content/drive/MyDrive/Data Science/customer_segmentation_train.csv', index_col=0)

# Перевірка на пропущені значення
print(raw_df.isnull().sum())

# Розділення даних на вхідні та цільові змінні
X = raw_df.drop('Segmentation', axis=1)
y = raw_df['Segmentation']

# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Виявлення числових і категоріальних колонок
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(include=[object]).columns

# Обробка числових колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Обробка категоріальних колонок
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Комбінування трансформерів для числових та категоріальних колонок в один препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення пайплайну
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Тренування пайплайну на тренувальних даних
X_train_preprocessed = model_pipeline.fit_transform(X_train)
X_test_preprocessed = model_pipeline.transform(X_test)

# Перевірка оброблених даних
print("Processed training data shape:", X_train_preprocessed.shape)
print("Processed test data shape:", X_test_preprocessed.shape)

# Визначення індексів категоріальних ознак після OneHotEncoding
encoder = model_pipeline.named_steps['preprocessor'].named_transformers_['cat']['onehot']
cat_feature_indices = list(range(len(numeric_features), len(numeric_features) + len(encoder.get_feature_names_out())))

# Застосування SMOTENC на всіх ознаках
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train_preprocessed, y_train)

print("SMOTENC results:")
print(X_train_smotenc.shape, y_train_smotenc.shape)


Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64
Processed training data shape: (6454, 28)
Processed test data shape: (1614, 28)
SMOTENC results:
(7176, 28) (7176,)


**Завдання 3.**

Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.
Виміряйте якість кожної з натренованих моделей використовуючи sklearn.metrics.classification_report.
Напишіть, яку метрику ви обрали для порівняння моделей.
Яка модель найкраща?
Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Завантаження даних
raw_df = pd.read_csv('/content/drive/MyDrive/Data Science/customer_segmentation_train.csv', index_col=0)

# Перевірка на пропущені значення
print(raw_df.isnull().sum())

# Розділення даних на вхідні та цільові змінні
X = raw_df.drop('Segmentation', axis=1)
y = raw_df['Segmentation']

# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Виявлення числових і категоріальних колонок
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(include=[object]).columns

# Обробка числових колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Обробка категоріальних колонок
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Комбінування трансформерів для числових та категоріальних колонок в один препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення пайплайну
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Тренування пайплайну на тренувальних даних
X_train_preprocessed = model_pipeline.fit_transform(X_train)
X_test_preprocessed = model_pipeline.transform(X_test)

# Перевірка оброблених даних
print("Processed training data shape:", X_train_preprocessed.shape)
print("Processed test data shape:", X_test_preprocessed.shape)

# Визначення індексів категоріальних ознак після OneHotEncoding
encoder = model_pipeline.named_steps['preprocessor'].named_transformers_['cat']['onehot']
cat_feature_indices = list(range(len(numeric_features), len(numeric_features) + len(encoder.get_feature_names_out())))

# Застосування SMOTENC на всіх ознаках
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train_preprocessed, y_train)

# Застосування SMOTE-Tomek на всіх ознаках
smote_tomek = SMOTETomek(random_state=42)
X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train_preprocessed, y_train)

print("Resampling applied on training data:")
print("Original training data shape:", X_train_preprocessed.shape, y_train.shape)
print("SMOTENC training data shape:", X_train_smotenc.shape, y_train_smotenc.shape)
print("SMOTE-Tomek training data shape:", X_train_smote_tomek.shape, y_train_smote_tomek.shape)


Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64
Processed training data shape: (6454, 28)
Processed test data shape: (1614, 28)
Resampling applied on training data:
Original training data shape: (6454, 28) (6454,)
SMOTENC training data shape: (7176, 28) (7176,)
SMOTE-Tomek training data shape: (5726, 28) (5726,)


In [ ]:
def train_and_evaluate(X_train, y_train, X_test, y_test):
    log_reg = LogisticRegression(solver='liblinear')
    ovr_model = OneVsRestClassifier(log_reg)
    ovr_model.fit(X_train, y_train)
    y_pred = ovr_model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    return report

# Оригінальні дані
print("Original Data:")
original_report = train_and_evaluate(X_train_preprocessed, y_train, X_test_preprocessed, y_test)
print(original_report)

# SMOTENC дані
print("SMOTENC Data:")
smotenc_report = train_and_evaluate(X_train_smotenc, y_train_smotenc, X_test_preprocessed, y_test)
print(smotenc_report)

# SMOTE-Tomek дані
print("SMOTE-Tomek Data:")
smote_tomek_report = train_and_evaluate(X_train_smote_tomek, y_train_smote_tomek, X_test_preprocessed, y_test)
print(smote_tomek_report)


Original Data:
{'A': {'precision': 0.39751552795031053, 'recall': 0.49104859335038364, 'f1-score': 0.43935926773455375, 'support': 391}, 'B': {'precision': 0.38345864661654133, 'recall': 0.13821138211382114, 'f1-score': 0.20318725099601592, 'support': 369}, 'C': {'precision': 0.45109780439121755, 'recall': 0.5947368421052631, 'f1-score': 0.5130533484676503, 'support': 380}, 'D': {'precision': 0.6659959758551308, 'recall': 0.6983122362869199, 'f1-score': 0.6817713697219362, 'support': 474}, 'accuracy': 0.49566294919454773, 'macro avg': {'precision': 0.47451698870330006, 'recall': 0.4805772634640969, 'f1-score': 0.459342809230039, 'support': 1614}, 'weighted avg': {'precision': 0.48576460362705687, 'recall': 0.49566294919454773, 'f1-score': 0.47390673541985456, 'support': 1614}}
SMOTENC Data:
{'A': {'precision': 0.4024640657084189, 'recall': 0.5012787723785166, 'f1-score': 0.4464692482915718, 'support': 391}, 'B': {'precision': 0.4039408866995074, 'recall': 0.2222222222222222, 'f1-score'

Висновок:
Усі три моделі показують дуже схожі результати. Метрики незначно відрізняються, але можна зробити висновок, що модель з даними, збалансованими за допомогою SMOTENC, показала трохи кращі результати за середнім F1-score.
Гіпотеза : Можливо, що самі дані мають такі особливості, що моделі не можуть навчитися добре розділяти класи навіть після балансування.
Також можливо логістична регресія не найкращий варіант застосування для цих даних.